# Chapter 12: Hash Tables

## Notes

* Given a corpus of `n` words with `m` being the maximum length of any word, the list of anagrams can be generated in `O(nm)` time as below. The chances of overflow can be reduced by reducing the probability of `hash` being a large value. This can be done by assigning lower primes to the more frequently used characters. Also look at this [link](https://stackoverflow.com/questions/11108541/get-list-of-anagrams-from-a-dictionary). 

```
procedure anagrams(corpus):
    - dict := Empty HashMap
    - For word in corpus:
        - dict[anagram_hash(word)] := append "word"
    - Return values of "dict" whose length is atleast 2
    
procedure anagram_hash(word):
    - prime_array := array of the first 26 prime numbers
    - hash := 1
    - For char in word:
        - hash := hash * (ascii(char) - ascii(`a`))
    - Return hash
```

* `set.remove(x)` vs `set.discard(x)`? The former raises a `KeyError` if `x` is not in set. The latter returns `None` instead.

* The built-in `hash()` function can greatly simplify the implementation of a hash function for a user-defined class i.e implementing `__hash__(self)`.

* `frozenset` is hashable alternative to `set` if a collection of non-duplicate elements need to be hashed.

## 12.1 Test for palindromic permutations

In [8]:
from collections import Counter

def can_form_palindrome(s):
    """
    Returns True iff the given string is a palindrome
    """
    return sum(v%2 for v in Counter(s).values()) <= 1

# Tests
assert not can_form_palindrome("hakuna")
assert can_form_palindrome("edified")

# 12.2 Is an anonymous letter constructible?

In [13]:
def is_letter_constructible(letter, magazine):
    """
    Returns True iff the given letter can be constructed from the given magazine
    """
    letter_count = Counter(letter)
    mag_count = Counter(magazine)
    return not letter_count - mag_count
    

# Tests
assert is_letter_constructible("hey", "hasenasdfy")
assert not is_letter_constructible("heyz", "hasenasdfy")
assert not is_letter_constructible("hhh", "hasdfas")

In [20]:
def is_letter_constructible_2(letter, magazine):
    letter_count = Counter(letter)
    for c in magazine:
        if c in letter_count:
            letter_count[c] -= 1
    return sum(v for v in letter_count.values() if v >= 0) == 0
    
    
# Tests
assert is_letter_constructible_2("hey", "hasenasdfy")
assert not is_letter_constructible_2("heyz", "hasenasdfy")
assert not is_letter_constructible_2("hhh", "hasdfas")
assert not is_letter_constructible_2("az", "aaay")

# 12.3 Implement an ISBN cache

In [21]:
from collections import OrderedDict

class ISBNCache:
    def __init__(self, capacity):
        self._cache = OrderedDict()
        self._capacity = capacity
        
    def lookup(self, isbn):
        if isbn not in self._cache:
            raise KeyError("ISBN {} does not exist".format(isbn))
        # The below pop + insert operation, puts the key-value pair at the end of the OrderedDict
        price = self._cache.pop(isbn)
        self._cache[isbn] = price
        return price
    
    def insert(self, isbn, price):
        if isbn in price:
            price = self._cache.pop(isbn)
        elif len(self._cache) >= self._capacity:
            # Remove LRU item. last = False assumes a FIFO ordering
            self._cache.popitem(last=False)
        self._cache[isbn] = price
    
    def delete(self, isbn):
        return self._cache.pop(isbn, None) is not None

In [ ]:
# Generic LRU Cache without using OrderedDict. 
# Other implementations: https://discuss.leetcode.com/topic/14591/python-dict-double-linkedlist/9

class Node:
    def __init__(self, k, v):
        self.key = k
        self.val = v
        self.prev = None
        self.next = None

class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.dic = dict()
        self.prev = self.next = self
        
    def get(self, key):
        if key in self.dic:
            n = self.dic[key]
            self._remove(n)
            self._add(n)
            return n.val
        return -1

    def put(self, key, value):
        if key in self.dic:
            self._remove(self.dic[key])
        n = Node(key, value)
        self._add(n)
        self.dic[key] = n
        if len(self.dic) > self.capacity:
            n = self.next
            self._remove(n)
            del self.dic[n.key]

    def _remove(self, node):
        p = node.prev
        n = node.next
        p.next = n
        n.prev = p

    def _add(self, node):
        p = self.prev
        p.next = node
        self.prev = node
        node.prev = p
        node.next = self